In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from prophet import Prophet
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
import warnings
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (20,7)

In [2]:
def plot_regions(var):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Piemonte"], line={'color': 'red'}, name="Piemonte"))
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Emilia-Romagna"], line={'color': 'blue'}, name="Emilia-Romagna"))
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Umbria"], line={'color': 'green'}, name="Umbria"))
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Puglia"], line={'color': 'orange'}, name="Puglia"))
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Sicilia"], line={'color': 'cyan'}, name="Sicilia"))
    fig.add_trace(go.Scatter(x=var.index, y=var.loc[:, "Sardegna"], line={'color': 'black'}, name="Sardegna"))
    fig.update_layout(title_text="Evapotranspiration")
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="mm/month")
    fig.show()

def split_train_test(data):
    perc = 0.9
    data_length = len(data)
    split_index = int(perc * data_length)
    if type(data) is pd.DataFrame:
        train = data.iloc[:split_index, :]
        test = data.iloc[split_index:, :]
    else:
        train = data.iloc[:split_index]
        test = data.iloc[split_index:]
    return train, test

In [3]:
def model_estimation(df, reg, flag_arima, flag_sarima, flag_prophet, flag_greykite):
    entire_ts = df[reg]
    train, test = split_train_test(entire_ts)
    plot_name = 'Evapotranspiration in ' + reg
    yaxes_name = 'mm/month'
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test.index, y=test, line={'color': 'black'}, name='Real values'))
    print("======\n")
    print("Analysis for " + reg + "\n")
    
    # ARIMA model
    if flag_arima == True:
        arima = pm.auto_arima(train, seasonal=False, error_action="ignore")
        arima_forecast = arima.predict(n_periods=len(test))
        fig.add_trace(go.Scatter(x=test.index, y=arima_forecast, line={'color': 'orange'}, name='Predicted points (ARIMA)'))
        error_arima = [mean_absolute_error(test, arima_forecast), mean_squared_error(test, arima_forecast)]
        print("ARIMA Model: " + str(arima.summary()).split("Model:")[1].split("Log")[0].strip())
    else:
        error_arima = [np.nan, np.nan]
    
    # SARIMA model
    if flag_sarima == True:
        sarima = pm.auto_arima(train, seasonal=True, m=12, error_action="ignore")
        sarima_forecast = sarima.predict(n_periods=len(test))
        fig.add_trace(go.Scatter(x=test.index, y=sarima_forecast, line={'color': 'red'}, name='Predicted points (SARIMA)'))
        error_sarima = [mean_absolute_error(test, sarima_forecast), mean_squared_error(test, sarima_forecast)]
        print("SARIMA Model: " + str(sarima.summary()).split("Model:")[1].split("Log")[0].strip())
    else:
        error_sarima = [np.nan, np.nan]
    
    # Prophet model
    if flag_prophet == True:
        m = Prophet()
        prophet_train = train.reset_index().rename(columns={'month': 'ds', train.name: 'y'})
        m.fit(prophet_train)
        future = m.make_future_dataframe(periods=len(test), freq=train.index.freq)
        prophet_forecast = m.predict(future).loc[:, ['ds', 'yhat', 'yhat_lower', 'yhat_upper']].set_index('ds')  
        fig.add_trace(go.Scatter(x=prophet_forecast.loc[test.index[0]:, :].index, y=prophet_forecast.loc[test.index[0]:, 'yhat'], line={'color': 'green'}, name='Predicted points (Prophet)'))
        fig.add_trace(go.Scatter(x=prophet_forecast.loc[test.index[0]:, :].index, y=prophet_forecast.loc[test.index[0]:, 'yhat_upper'], line={'color':'lightgreen', 'dash':'dash'}, name='Predicted upper confidence (Prophet)', visible="legendonly"))
        fig.add_trace(go.Scatter(x=prophet_forecast.loc[test.index[0]:, :].index, y=prophet_forecast.loc[test.index[0]:, 'yhat_lower'], line={'color':'lightgreen', 'dash':'dash'}, name='Predicted lower confidence (Prophet)', visible="legendonly"))
        error_prophet = [mean_absolute_error(test, prophet_forecast.loc[test.index[0]:, 'yhat']), mean_squared_error(test, prophet_forecast.loc[test.index[0]:, 'yhat'])]
    else:
        error_prophet = [np.nan, np.nan]
    
    # Greykite model
    if flag_greykite == True:
        forecaster = Forecaster()
        greykite_train = train.reset_index().rename(columns={'month': 'ts', train.name: 'y'})
        metadata = MetadataParam(time_col="ts", value_col="y", freq="MS")
        result = forecaster.run_forecast_config(df=greykite_train, config=ForecastConfig(model_template=ModelTemplateEnum.SILVERKITE.name, forecast_horizon=len(test), coverage=0.95, metadata_param=metadata))
        greykite_forecast = result.forecast.df
        greykite_forecast = greykite_forecast[np.isnan(greykite_forecast.actual)]
        fig.add_trace(go.Scatter(x=greykite_forecast.ts, y=greykite_forecast.forecast, line={'color': 'blue'}, name="Predicted points (Greykite)"))
        fig.add_trace(go.Scatter(x=greykite_forecast.ts, y=greykite_forecast.forecast_lower, line={'color': 'cyan', 'dash':'dash'}, name="Predicted lower confidence (Greykite)", visible="legendonly"))
        fig.add_trace(go.Scatter(x=greykite_forecast.ts, y=greykite_forecast.forecast_upper, line={'color': 'cyan', 'dash':'dash'}, name="Predicted upper confidence (Greykite)", visible="legendonly"))
        error_greykite = [mean_absolute_error(test, greykite_forecast.forecast), mean_squared_error(test, greykite_forecast.forecast)]
    else:
        error_greykite = [np.nan, np.nan]

    fig.add_trace(go.Scatter(x=train.index, y=train, line={'color': 'grey'}, name='Train', visible="legendonly"))
    fig.update_layout(title_text = plot_name)
    fig.update_xaxes(title_text = 'Date')
    fig.update_yaxes(title_text = yaxes_name)
    fig.show()
    print("\n\nMAE/RMSE of ARIMA model: " + str(error_arima))
    print("\nMAE/RMSE of SARIMA model: " + str(error_sarima))
    print("\nMAE/RMSE of Prophet model: " + str(error_prophet))
    print("\nMAE/RMSE of Greykite model: " + str(error_greykite))
    return pd.DataFrame({"region": reg,"error_ARIMA": [error_arima], "error_SARIMA": [error_sarima], "error_Prophet": [error_prophet], "error_Greykite": [error_greykite]})

In [4]:
ae_all = pd.read_csv("output/ae.csv", parse_dates=["month"], index_col="month")
tp_all = pd.read_csv("output/tp.csv", parse_dates=["month"], index_col="month")
regions = ["Piemonte", "Emilia-Romagna", "Umbria", "Puglia", "Sicilia", "Sardegna"]
# regions = ["Piemonte", "Valle d\'Aosta", "Lombardia", "Trentino-Alto Adige", "Veneto", "Friuli Venezia Giulia", "Liguria", "Emilia-Romagna", "Toscana", "Umbria", "Marche", "Lazio", "Abruzzo", "Molise", "Campania", "Puglia", "Basilicata", "Calabria", "Sicilia", "Sardegna"]
# regions = ["Piemonte"]
ae = ae_all.filter(regions, axis=1)
tp = tp_all.filter(regions, axis=1)
ae.index.freq = ae.index.inferred_freq
tp.index.freq = tp.index.inferred_freq
ae_0 = ae[:12]
ae = ae.diff(12)[12:]
tp_0 = tp[:12]
tp = tp.diff(12)[12:]
plot_regions(ae)
plot_regions(tp)

### Evapotranspiration

In [5]:
errors_ae = pd.DataFrame(columns=["region","error_ARIMA", "error_SARIMA", "error_Prophet", "error_Greykite"])

for region in regions:
    temp = model_estimation(ae, region, True, True, True, True)
    errors_ae = errors_ae.append(temp)


Analysis for Piemonte

ARIMA Model: SARIMAX(0, 0, 2)
SARIMA Model: SARIMAX(2, 0, 1)x(2, 0, [], 12)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [10.103867624088675, 200.80145214806322]

MAE/RMSE of SARIMA model: [9.288713119777624, 174.25776744171003]

MAE/RMSE of Prophet model: [10.134245752706612, 201.53554227050145]

MAE/RMSE of Greykite model: [10.089360174726314, 200.38097976203576]

Analysis for Emilia-Romagna

ARIMA Model: SARIMAX(1, 0, 0)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

SARIMA Model: SARIMAX(1, 0, 0)x(2, 0, 0, 12)
Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [11.811476892141677, 353.88915455537085]

MAE/RMSE of SARIMA model: [11.538991372125928, 344.86363436202504]

MAE/RMSE of Prophet model: [11.808194221106156, 354.1800146127904]

MAE/RMSE of Greykite model: [11.813403384868735, 353.84405424808523]

Analysis for Sardegna

ARIMA Model: SARIMAX(1, 0, 2)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


SARIMA Model: SARIMAX(1, 0, 0)x(2, 0, 0, 12)
Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [14.450638289386156, 499.1785709332466]

MAE/RMSE of SARIMA model: [13.96081372438841, 480.1482953171536]

MAE/RMSE of Prophet model: [14.887903406679625, 537.1481708037053]

MAE/RMSE of Greykite model: [14.481528663586412, 499.28394677740624]


In [6]:
errors_ae.reset_index(drop=True)
errors_ae["MSE SARIMA/Prophet [%]"] = errors_ae.error_SARIMA.str[0] / errors_ae.error_Prophet.str[0] * 100 - 100
errors_ae["RMSE SARIMA/Prophet [%]"] = errors_ae.error_SARIMA.str[1] / errors_ae.error_Prophet.str[1] * 100 - 100
errors_ae["MSE SARIMA/GreyKite [%]"] = errors_ae.error_SARIMA.str[0] / errors_ae.error_Greykite.str[0] * 100 - 100
errors_ae["RMSE SARIMA/GreyKite [%]"] = errors_ae.error_SARIMA.str[1] / errors_ae.error_Greykite.str[1] * 100 - 100
errors_ae = errors_ae.sort_values(by="RMSE SARIMA/Prophet [%]", ascending=False).reset_index(drop=True)
errors_ae

,region,error_ARIMA,error_SARIMA,error_Prophet,error_Greykite,MSE SARIMA/Prophet [%],RMSE SARIMA/Prophet [%],MSE SARIMA/GreyKite [%],RMSE SARIMA/GreyKite [%]
0,Puglia,"[14.760977818535105, 437.07090311172993]","[14.641491927978445, 439.2956350969074]","[14.720110508738333, 436.79146198011193]","[14.737467760800335, 436.88850943051415]",-0.534090,0.573311,-0.651237,0.550970
1,Sicilia,"[11.811476892141677, 353.88915455537085]","[11.538991372125928, 344.86363436202504]","[11.808194221106156, 354.1800146127904]","[11.813403384868735, 353.84405424808523]",-2.279797,-2.630408,-2.322887,-2.537960
2,Umbria,"[15.029767207487813, 480.7373689640851]","[14.712020127921749, 441.44484035491894]","[15.082494234393616, 481.66495524471577]","[15.034928197461653, 480.6581985479771]",-2.456319,-8.350227,-2.147719,-8.158263
3,Sardegna,"[14.450638289386156, 499.1785709332466]","[13.96081372438841, 480.1482953171536]","[14.887903406679625, 537.1481708037053]","[14.481528663586412, 499.28394677740624]",-6.227134,-10.611574,-3.595718,-3.832619
4,Emilia-Romagna,"[13.478905579846838, 361.4997263723028]","[12.378007966883102, 319.2082326130378]","[13.532174122208788, 362.99228061940386]","[13.479299064076578, 361.41428080041146]",-8.529052,-12.061978,-8.170240,-11.678024
5,Piemonte,"[10.103867624088675, 200.80145214806322]","[9.288713119777624, 174.25776744171003]","[10.134245752706612, 201.53554227050145]","[10.089360174726314, 200.38097976203576]",-8.343321,-13.534970,-7.935558,-13.036772


### Precipitation

In [7]:
errors_pt = pd.DataFrame(columns=["region","error_ARIMA", "error_SARIMA", "error_Prophet", "error_Greykite"])

for region in regions:
    temp = model_estimation(tp, region, True, True, True, True) 
    errors_pt = errors_pt.append(temp)


Analysis for Piemonte

ARIMA Model: SARIMAX(1, 0, 0)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


SARIMA Model: SARIMAX(2, 0, 0, 12)
Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [61.20418974481723, 8101.2700604616075]

MAE/RMSE of SARIMA model: [61.99874490914265, 7975.617354318226]

MAE/RMSE of Prophet model: [61.32212878629063, 8132.42151833833]

MAE/RMSE of Greykite model: [61.213706636008226, 8098.912996405321]

Analysis for Emilia-Romagna

ARIMA Model: SARIMAX


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

SARIMA Model: SARIMAX(0, 0, 1)x(2, 0, [], 12)
Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [37.92070834729553, 2684.5081723590406]

MAE/RMSE of SARIMA model: [38.20136860370901, 2710.194826624192]

MAE/RMSE of Prophet model: [37.659579337900176, 2674.256387537084]

MAE/RMSE of Greykite model: [37.77405027103977, 2675.2995358800094]

Analysis for Sardegna

ARIMA Model: SARIMAX


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


SARIMA Model: SARIMAX(2, 0, 0, 12)
Fitting 3 folds for each of 1 candidates, totalling 3 fits




MAE/RMSE of ARIMA model: [43.427607739483044, 3536.655547879483]

MAE/RMSE of SARIMA model: [42.24425559581684, 3266.1272094850633]

MAE/RMSE of Prophet model: [43.33929048787885, 3584.7432371130717]

MAE/RMSE of Greykite model: [43.449613250527754, 3536.235687834765]


In [8]:
errors_pt.reset_index(drop=True)
errors_pt["MSE SARIMA/Prophet [%]"] = errors_pt.error_SARIMA.str[0] / errors_pt.error_Prophet.str[0] * 100 - 100
errors_pt["RMSE SARIMA/Prophet [%]"] = errors_pt.error_SARIMA.str[1] / errors_pt.error_Prophet.str[1] * 100 - 100
errors_pt["MSE SARIMA/GreyKite [%]"] = errors_pt.error_SARIMA.str[0] / errors_pt.error_Greykite.str[0] * 100 - 100
errors_pt["RMSE SARIMA/GreyKite [%]"] = errors_pt.error_SARIMA.str[1] / errors_pt.error_Greykite.str[1] * 100 - 100
errors_pt = errors_pt.sort_values(by="RMSE SARIMA/Prophet [%]", ascending=False).reset_index(drop=True)
errors_pt

,region,error_ARIMA,error_SARIMA,error_Prophet,error_Greykite,MSE SARIMA/Prophet [%],RMSE SARIMA/Prophet [%],MSE SARIMA/GreyKite [%],RMSE SARIMA/GreyKite [%]
0,Umbria,"[49.991557927055034, 3942.439544198255]","[50.86851025812797, 4198.58618984464]","[49.840572117147495, 3954.5541527226833]","[49.896104521681636, 3945.762860473953]",2.062453,6.170912,1.948861,6.407464
1,Puglia,"[39.018227566858044, 2224.652784738862]","[39.15488340740913, 2287.989652775132]","[38.894546449305295, 2226.1883854211405]","[38.99895594864002, 2224.794487726365]",0.669341,2.776102,0.399825,2.840494
2,Sicilia,"[37.92070834729553, 2684.5081723590406]","[38.20136860370901, 2710.194826624192]","[37.659579337900176, 2674.256387537084]","[37.77405027103977, 2675.2995358800094]",1.438649,1.343867,1.131248,1.304351
3,Emilia-Romagna,"[47.39603411489603, 3635.9479827137434]","[47.66658136370991, 3656.8018540553685]","[47.58178808947405, 3661.6171251679893]","[47.387627949826985, 3644.7140350381806]",0.178205,-0.131507,0.588663,0.331653
4,Piemonte,"[61.20418974481723, 8101.2700604616075]","[61.99874490914265, 7975.617354318226]","[61.32212878629063, 8132.42151833833]","[61.213706636008226, 8098.912996405321]",1.103380,-1.928136,1.282455,-1.522373
5,Sardegna,"[43.427607739483044, 3536.655547879483]","[42.24425559581684, 3266.1272094850633]","[43.33929048787885, 3584.7432371130717]","[43.449613250527754, 3536.235687834765]",-2.526656,-8.888113,-2.774150,-7.638305
